In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install pyg-lib -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

2.2.1+cu121
Looking in links: https://data.pyg.org/whl/torch-2.2.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.8 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-nffq0voj
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-nffq0voj
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit e213c297bb2aeb9ac50db258f5ab01ea11aea349
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.6.0-py3-none-any.whl size=1104721 sha256=378c6de9d0c979d123852fee1982397f0d5f9855cb821e6d07eee799b7d26bab
  Stored in directory: /tmp/pip-ephem-wheel-cache-drvehm23/wheels/d3/78/eb/9e26525b948d19533f1688fb6c209cec8a0ba793d39b49ae8f
Successfully built torch_geometric


In [2]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

In [3]:
dataset = Planetoid(root='./datasets/Planetoid', name = 'PubMed', transform=NormalizeFeatures())

In [6]:
dataset
dataset[0]
dataset.num_classes
dataset.num_edge_features
dataset.num_node_features

PubMed()

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])

3

0

500

In [9]:
data = dataset[0]
data
data.num_nodes
data.num_edges /  data.num_nodes
data['train_mask'].sum().item() / data.num_nodes

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])

19717

4.496018664096972

0.0030430592889384793

In [10]:
from torch_geometric.loader import ClusterData, ClusterLoader

In [11]:
cluster_data = ClusterData(data, num_parts=128)
train_loader = ClusterLoader(cluster_data, batch_size = 32, shuffle = True)

Computing METIS partitioning...
Done!


In [15]:
cluster_data
cluster_data.data
cluster_data.keep_inter_cluster_edges
cluster_data.num_parts
cluster_data.partition
cluster_data.sparse_format

ClusterData(128)

Data(x=[19717, 500], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])

False

128

Partition(indptr=tensor([    0,     2,     3,  ..., 88645, 88647, 88648]), index=tensor([    1,   151,     0,  ..., 18371, 19681, 19613]), partptr=tensor([    0,   154,   308,   462,   616,   769,   923,  1077,  1231,  1385,
         1539,  1693,  1847,  2001,  2156,  2310,  2465,  2620,  2769,  2927,
         3080,  3236,  3389,  3545,  3701,  3854,  4007,  4160,  4315,  4469,
         4622,  4777,  4930,  5084,  5238,  5392,  5546,  5700,  5854,  6008,
         6162,  6315,  6469,  6623,  6777,  6931,  7084,  7238,  7392,  7546,
         7700,  7853,  8007,  8160,  8314,  8467,  8625,  8779,  8933,  9086,
         9240,  9395,  9549,  9703,  9857, 10011, 10165, 10320, 10474, 10628,
        10782, 10936, 11089, 11243, 11397, 11551, 11705, 11859, 12013, 12166,
        12320, 12474, 12626, 12780, 12934, 13088, 13242, 13396, 13550, 13704,
        13858, 14012, 14166, 14321, 14476, 14630, 14785, 14939, 15093, 15247,
        15401, 15555, 15709, 15862, 16016, 16171, 16326, 16481, 16635, 16

'csr'

In [17]:
total_num_nodes = 0
for step, sub_data in enumerate(train_loader):
    print(f"step :: {step + 1}")
    print(f"num of nodes :: {sub_data.num_nodes}")
    print(sub_data)
    total_num_nodes = total_num_nodes + sub_data.num_nodes

step :: 1
num of nodes :: 4933
Data(x=[4933, 500], y=[4933], train_mask=[4933], val_mask=[4933], test_mask=[4933], edge_index=[2, 16720])
step :: 2
num of nodes :: 4929
Data(x=[4929, 500], y=[4929], train_mask=[4929], val_mask=[4929], test_mask=[4929], edge_index=[2, 16538])
step :: 3
num of nodes :: 4923
Data(x=[4923, 500], y=[4923], train_mask=[4923], val_mask=[4923], test_mask=[4923], edge_index=[2, 16942])
step :: 4
num of nodes :: 4932
Data(x=[4932, 500], y=[4932], train_mask=[4932], val_mask=[4932], test_mask=[4932], edge_index=[2, 16160])


In [23]:
total_num_nodes
(total_num_nodes / 128) * 32 # num nodes per batch for batch_size 32

19717

4929.25

In [20]:
sub_data
sub_data['x']

Data(x=[4932, 500], y=[4932], train_mask=[4932], val_mask=[4932], test_mask=[4932], edge_index=[2, 16160])

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0394, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0343, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0103, 0.0157, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0577, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0095, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [24]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

In [27]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels=dataset.num_node_features, out_channels=hidden_channels)
        self.conv2 = GCNConv(in_channels=hidden_channels, out_channels=dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [28]:
model= GCN(hidden_channels=16)
model

GCN(
  (conv1): GCNConv(500, 16)
  (conv2): GCNConv(16, 3)
)

In [29]:
optimizer = torch.optim.Adam(params = model.parameters(), lr = 0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

In [30]:
def train():
    model.train()
    for sub_data in train_loader:
        optimizer.zero_grad()
        out = model(sub_data['x'], sub_data['edge_index'])
        loss = criterion(out[sub_data['train_mask']], sub_data['y'][sub_data['train_mask']])
        loss.backward()
        optimizer.step()

def test():
    model.eval()
    out = model(data['x'], data['edge_index'])
    pred = out.argmax(dim=1)

    acc = []
    for mask in [data['train_mask'], data['val_mask'], data['test_mask']]:
        correct = pred[mask] == data['y'][mask]
        acc.append(int(correct.sum()) / int(mask.sum()))
    return acc

In [31]:
for epoch in range(1,101):
    loss = train()
    if epoch % 10 == 0:
        train_acc, val_acc, test_acc = test()
        print(f"epoch = {epoch} :: train_acc = {train_acc} :: val_acc = {val_acc}, test_acc = {test_acc}")

epoch = 10 :: train_acc = 0.9333333333333333 :: val_acc = 0.764, test_acc = 0.733
epoch = 20 :: train_acc = 0.9666666666666667 :: val_acc = 0.788, test_acc = 0.757
epoch = 30 :: train_acc = 0.9833333333333333 :: val_acc = 0.796, test_acc = 0.766
epoch = 40 :: train_acc = 0.9833333333333333 :: val_acc = 0.8, test_acc = 0.785
epoch = 50 :: train_acc = 0.9833333333333333 :: val_acc = 0.812, test_acc = 0.782
epoch = 60 :: train_acc = 0.9833333333333333 :: val_acc = 0.798, test_acc = 0.781
epoch = 70 :: train_acc = 0.9833333333333333 :: val_acc = 0.8, test_acc = 0.773
epoch = 80 :: train_acc = 0.9833333333333333 :: val_acc = 0.808, test_acc = 0.781
epoch = 90 :: train_acc = 1.0 :: val_acc = 0.8, test_acc = 0.776
epoch = 100 :: train_acc = 1.0 :: val_acc = 0.798, test_acc = 0.778
